In [1]:
from langchain.tools import tool 
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
import requests 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Tool Create 
from langchain_core.tools import InjectedToolArg 
from typing import Annotated 

@tool 
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This function fetches the currency conversion factor between a given base currency and target currency"""
    url = f"https://v6.exchangerate-api.com/v6/0c0ce63945d4b33b1e4326a8/pair/{base_currency}/{target_currency}"

    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Given a currency conversion rate this function calculates the target currency value from a given base currency value"""
    return base_currency_value * conversion_rate

In [3]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1752883201,
 'time_last_update_utc': 'Sat, 19 Jul 2025 00:00:01 +0000',
 'time_next_update_unix': 1752969601,
 'time_next_update_utc': 'Sun, 20 Jul 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 86.1694}

In [4]:
convert.invoke({'base_currency_value':10, 'conversion_rate': 86.1694})

861.694

#### **Tool Binding**

In [5]:
llm = ChatOpenAI()

llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

#### **Tool Calling**

In [15]:
messages = [HumanMessage("What is the conversion factor of USD and INR, and based on that convert 500 USD to INR")]
messages

[HumanMessage(content='What is the conversion factor of USD and INR, and based on that convert 500 USD to INR', additional_kwargs={}, response_metadata={})]

In [16]:
ai_message = llm_with_tools.invoke(messages)

In [17]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_0ltS2YggKPosCwZEAdXdksK9',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 500},
  'id': 'call_qNIkJOGTf0N6jO0hRdCv6y5O',
  'type': 'tool_call'}]

In [18]:
messages.append(ai_message)

In [19]:
import json 

for tool_call in ai_message.tool_calls:
    # Execute the 1st tool and get value of conversion rate
    if tool_call['name'] == "get_conversion_factor":
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # print(tool_message1)
        # Fetch the conversion rate 
        conversion_rate = json.loads(tool_message1.content)["conversion_rate"]

        # Append this tool message to messages list
        messages.append(tool_message1)

    # Execute the 2nd tool using the conversion rate from tool 1
    if tool_call["name"] == "convert":
        # Fetch the current argument
        tool_call["args"]["conversion_rate"] = conversion_rate

        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [20]:
messages

[HumanMessage(content='What is the conversion factor of USD and INR, and based on that convert 500 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0ltS2YggKPosCwZEAdXdksK9', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_qNIkJOGTf0N6jO0hRdCv6y5O', 'function': {'arguments': '{"base_currency_value": 500}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 119, 'total_tokens': 171, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bv4VCoK3agCWbC1oMRcYsbmTICngv', 'service_tier': 'default', 'fini

In [21]:
print(llm_with_tools.invoke(messages).content)

The conversion factor from USD to INR is 86.1694. 

Based on this, converting 500 USD to INR gives 43,084.7 INR.
